In [58]:
def _rouge_score(start_y, end_y, start_pred, end_pred, gamma):
    """ 计算某一条记录的rouge """
    start = max(start_y, start_pred)
    end = min(end_y, end_pred)

    interval = end - start + 1
    if interval <= 0:
        return 0
    else:
        length_pred = end_pred - start_pred + 1
        length_y = end_y - start_y + 1
        prec = interval / length_pred if length_pred > 0 else 0
        rec = interval / length_y if length_y >0 else 0

        if prec !=0 and rec != 0:
            score = ((1 + gamma**2) * prec * rec) / (rec + gamma**2 * prec)
        else:
            score = 0
        return score

In [63]:
start_y = 4
end_y = 5

start_pred = 4
end_pred = 5
_rouge_score(start_y, end_y, start_pred, end_pred, 1)

1.0

In [62]:
range(2, 5)

range(2, 5)

In [65]:
a = torch.Tensor([1, 2, 3])

In [121]:
a = torch.stack([torch.arange(3), torch.arange(3)])
a

tensor([[ 0.,  1.,  2.],
        [ 0.,  1.,  2.]])

In [122]:
a.unsqueeze(2).expand(2, 3, 3)

tensor([[[ 0.,  0.,  0.],
         [ 1.,  1.,  1.],
         [ 2.,  2.,  2.]],

        [[ 0.,  0.,  0.],
         [ 1.,  1.,  1.],
         [ 2.,  2.,  2.]]])

In [123]:
a.unsqueeze(1).expand(2, 3, 3)

tensor([[[ 0.,  1.,  2.],
         [ 0.,  1.,  2.],
         [ 0.,  1.,  2.]],

        [[ 0.,  1.,  2.],
         [ 0.,  1.,  2.],
         [ 0.,  1.,  2.]]])

In [125]:
a.unsqueeze(1).expand(2, 3, 3) - a.unsqueeze(2).expand(2, 3, 3) 

tensor([[[ 0.,  1.,  2.],
         [-1.,  0.,  1.],
         [-2., -1.,  0.]],

        [[ 0.,  1.,  2.],
         [-1.,  0.,  1.],
         [-2., -1.,  0.]]])

In [89]:
b = torch.Tensor([3,2]).view(-1, 1)
b

tensor([[ 3.],
        [ 2.]])

In [111]:
a = torch.Tensor([[1,2,2], [4,5,1]])
a

tensor([[ 1.,  2.,  2.],
        [ 4.,  5.,  1.]])

In [112]:
torch.topk(a, k=5)

RuntimeError: invalid argument 2: k not in range for dimension at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/TH/generic/THTensorMath.c:3439

In [101]:
a

tensor([[ 1.,  2.,  2.],
        [ 4.,  5.,  1.]])

In [100]:
a.unsqueeze(1).expand(2, 3,3)

tensor([[[ 1.,  2.,  2.],
         [ 1.,  2.,  2.],
         [ 1.,  2.,  2.]],

        [[ 4.,  5.,  1.],
         [ 4.,  5.,  1.],
         [ 4.,  5.,  1.]]])

In [102]:
a.unsqueeze(2).expand(2,3,3)

tensor([[[ 1.,  1.,  1.],
         [ 2.,  2.,  2.],
         [ 2.,  2.,  2.]],

        [[ 4.,  4.,  4.],
         [ 5.,  5.,  5.],
         [ 1.,  1.,  1.]]])

In [104]:
b

tensor([[ 3.],
        [ 2.]])

In [105]:
b.unsqueeze(2).expand(2,3,3)

tensor([[[ 3.,  3.,  3.],
         [ 3.,  3.,  3.],
         [ 3.,  3.,  3.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]])

In [106]:
b.unsqueeze(1).expand(2,3,3)

tensor([[[ 3.,  3.,  3.],
         [ 3.,  3.,  3.],
         [ 3.,  3.,  3.]],

        [[ 2.,  2.,  2.],
         [ 2.,  2.,  2.],
         [ 2.,  2.,  2.]]])

In [108]:
torch.sum(b, dim=1)

tensor([ 3.,  2.])

In [110]:
a

tensor([ 1.,  2.,  3.])